In [11]:
# !python --version
# !tensorflow --version
# !pip install Path
# !pip install tensorflow
# !pip install opencv-python
# !pip install tensorflow-hub
# !pip install pydicom
#!pip install pandas
#!pip install matplotlib
#!pip install tqdm
#!pip install scikit-learn

     ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
     --------------------------------------- 0.0/42.2 MB 660.6 kB/s eta 0:01:04
     ---------------------------------------- 0.1/42.2 MB 1.1 MB/s eta 0:00:41
     ---------------------------------------- 0.3/42.2 MB 2.7 MB/s eta 0:00:16
      --------------------------------------- 0.9/42.2 MB 5.0 MB/s eta 0:00:09
     - -------------------------------------- 1.5/42.2 MB 7.2 MB/s eta 0:00:06
     -- ------------------------------------- 3.0/42.2 MB 11.1 MB/s eta 0:00:04
     --- ------------------------------------ 3.5/42.2 MB 12.5 MB/s eta 0:00:04
     ---- ----------------------------------- 4.5/42.2 MB 12.4 MB/s eta 0:00:04
     ----- ---------------------------------- 5.8/42.2 MB 14.8 MB/s eta 0:00:03
     ------- -------------------------------- 7.9/42.2 MB 17.3 MB/s eta 0:00:02
     ------- -------------------------------- 8.1/42.2 MB 16.2 MB/s eta 0:00:03
     --------- ----------------------------- 10.0/42

In [4]:



import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 as cv
from path import Path
import os 
import glob
import tensorflow_hub as hub
import os 
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import tensorflow as tf
from tensorflow import keras
from keras import layers
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix


In [5]:
train_df= pd.read_csv('C:\\ML Data\\BrainTumorML\\train_labels.csv')
sample_df = pd.read_csv('C:\\ML Data\\BrainTumorML\\sample_submission.csv')
#train_df.head(1)
#train_df.tail()
#train_df.sample(6)

In [6]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [7]:
def listdirs(folder):
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

In [19]:
#train_df = train_df[:100]

In [8]:
train_dir='C:\\ML Data\\BrainTumorML\\train'
trainset=[]
trainlabel=[]
trainidt=[]
for i in tqdm(range(len(train_df))):
    idt=train_df.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:]
    path=os.path.join(train_dir,idt2,'T1wCE')              
    for im in os.listdir(path):
        img=load_dicom(os.path.join(path,im)) 
        img=cv.resize(img,(64,64)) 
        image=img_to_array(img)
        image=image/255.0
        trainset+=[image]
        trainlabel+=[train_df.loc[i,'MGMT_value']]
        trainidt+=[idt]

100%|██████████| 585/585 [14:38<00:00,  1.50s/it]


In [9]:
test_dir='C:\\ML Data\\BrainTumorML\\test'
testset=[]
testidt=[]
for i in tqdm(range(len(sample_df))):
    idt=sample_df.loc[i,'BraTS21ID']
    idt2=('00000'+str(idt))[-5:]
    path=os.path.join(test_dir,idt2,'T1wCE')               
    for im in os.listdir(path):   
        img=load_dicom(os.path.join(path,im))
        img=cv.resize(img,(64,64)) 
        image=img_to_array(img)
        image=image/255.0
        testset+=[image]
        testidt+=[idt]
print(testidt[0])
#print("This is the first testIdt" , testidt[0])
firstEX = testidt[0]
print(f"This is the first testidt {firstEX}")

100%|██████████| 87/87 [02:07<00:00,  1.47s/it]

1
This is the first testidt 1


In [10]:
import pickle
pickle.dump(testidt, open('C:\\ML Data\\BrainTumorML\\test\\testIdt.label', 'wb'))

In [11]:
y=np.array(trainlabel)
Y_train=to_categorical(y)
X_train=np.array(trainset)
X_test=np.array(testset)

In [26]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model

path = "model_100.keras"


  # Create your model here or load it from somewhere else if it doesn't exist.
    # Assuming you have a model variable defined.
#hist = model.fit(X_train, Y_train, epochs=100, batch_size=64, verbose=1, callbacks=[callback])
#model.save("C:\\ML Data\\BrainTumorMLtrain\\model_100.keras")
    

if os.path.isfile(path):
    model = load_model(path)

if model is not None:
        if hasattr(model, 'history'):
            #get_ac = model.history['accuracy']
            #get_los = model.history['loss']
            loss, accuracy = model.evaluate(X_train, Y_train)
    
        else:
            print("Model does not have a history attribute.")
            model = keras.models.Sequential()
            model.add(keras.layers.Conv2D(filters=64,kernel_size=(4,4),input_shape=(64,64,1),activation='relu',kernel_initializer="he_normal"))
            model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Conv2D(filters=64,kernel_size=(4,4),activation='relu',kernel_initializer="he_normal"))
            model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
            model.add(keras.layers.Dropout(0.20))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Conv2D(filters=64,kernel_size=(4,4),activation='relu',kernel_initializer="he_normal"))
            model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
            model.add(keras.layers.Dropout(0.25))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Flatten())
            model.add(keras.layers.Dense(100,activation="relu",kernel_initializer="he_normal"))
            model.add(keras.layers.Dense(2,"softmax"))
            model.compile(loss="categorical_crossentropy", optimizer = "RMSprop",metrics=["accuracy"])
            callback = keras.callbacks.EarlyStopping(monitor='loss', patience=8)
            hist = model.fit(X_train, Y_train,epochs=100, batch_size=64, verbose=1,callbacks=[callback])
            model.save(path)
else: 
    print("Failed to load the model from the specified path.")
        
model.summary()
print("Successful")


# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(filters=64, kernel_size=(4,4), input_shape=(64,64,1), activation='relu', kernel_initializer="he_normal"))
# model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu', kernel_initializer="he_normal"))
# model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Dropout(0.20))
# model.add(keras.layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu', kernel_initializer="he_normal"))
# model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(keras.layers.Dropout(0.25))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(0.20))
# model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
# model.add(keras.layers.Dropout(0.25))
# model.add(keras.layers.Dense(1, activation="sigmoid"))


3024/3024 [==============================] - 45s 15ms/step - loss: 0.2139 - accuracy: 0.8696
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 61, 61, 64)        1088      
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 64)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 30, 30, 64)        256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 64)        65600     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                        